In [1]:
import subprocess
from faster_whisper import WhisperModel
# from lista import segmentos_lista
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
import os
import asyncio
import json
import re
from langchain_core.output_parsers import JsonOutputParser
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_together import ChatTogether

load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['TOGETHER_API_KEY'] = os.getenv('TOGETHER_API_KEY')

In [2]:
llm = ChatTogether(model="meta-llama/Llama-3-70b-chat-hf", temperature=0)

In [3]:
# llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")

In [4]:
# llm = ChatOllama(model="wizardlm2", temperature=0)

In [5]:
# llm = ChatOpenAI(base_url="http://localhost:1234/v1",api_key="lm-studio", model_name="cognitivecomputations/dolphin-2.9-llama3-8b-gguf/dolphin-2.9-llama3-8b-q6_K.gguf", temperature=0)

In [6]:
lista1 = [
    "Five levels of LLM apps. Consider this to be a framework and help you decide where you can",
    "use LLM. There are a lot of different myths around what LLMs can do, what LLMs cannot do.",
    "Where do you use LLMs today? So I decided to put together this material in which I'm going to take",
    "you through kind of like a mental framework based on the extension or the depth in which you go",
    "towards an LLM. You can decide where you can fit this LLM. So we're going to first see what are",
    "those different levels of LLMs that I have put together. Then we are going to see a slight",
    "extension of that. Got two different documents to take you through that. So this will give",
    "you an idea about how LLM is being used today and how you can use LLMs for your own applications.",
    "To start with, imagine this pyramid structure. This is a very simple pyramid structure",
    "and as you can imagine with any pyramid structure, the top of the pyramid or the",
    "peak of the pyramid is our aspirational goal. And what you see at the bottom is the easiest",
    "that we can do. And as with everything else, you have to slowly climb to the top of the pyramid.",
    "So you can probably hit the aspirational goal. So to start with, where do we use LLMs first? Q&A,",
    "a question and answering engine. What do I mean by that? It is quite simple for us to understand.",
    "So question and answering engine is a system where you have an LLM and all you are going to",
    "ask the LLM is a question. So you send a prompt and the LLM takes the prompt and gives you an",
    "answer. That is it. That is the entire transaction that you have between an LLM. Send a prompt,",
    "send it to the LLM, get an answer. LLM, large language models are nothing but sophisticated",
    "next word prediction engines. And they have been fine-tuned with something called instruction. So",
    "there are instruction fine-tuned models. That means they can take a human instruction and get",
    "you an answer back. For example, if I ask a question for this, what is the capital of India?",
    "Then the LLM would process this and then LLM has information about how to answer it. And then it",
    "will give me the answer back. The capital of India is New Delhi. That's all what you're going to do",
    "with this thing. So first level question and answering. Now you might wonder at this point",
    "that where can you use question and answering as an LLM engine? This is the first thing that people",
    "built like when LLM started, even back in the day, GP2 level, people started building simply",
    "Q&A bots. So all you want to do is ask a question, give an answer. Could be a homework, could be a",
    "general knowledge question, could be something about the world, could be about science, could be",
    "about anything. Ask a question, get an answer. As simple as that. It's a very three-step process.",
    "Ask a question or send a prompt, take the LLM to process it, give me the answer back.",
    "Very simple application. Now what you're going to do is you're going to add something to that",
    "application. And that is how you actually build a conversational chat bot. And to understand this",
    "better, I would like to take you to my second document, which will give you probably better",
    "idea. Whenever we are talking about LLM, there is one important thing that we need to understand is",
    "we have crossed the stage where LLM is simply a large language model. We have more than that.",
    "So for you to understand that, I have five dimensions, a prompt, a short-term memory,",
    "an external knowledge, tools, and extended tools. If you think of this as your horizontal,",
    "these are your verticals. These are different dimensions that can add to an LLM. So you have",
    "a prompt, you have a short-term memory, you have a long-term memory or external data, you have tools",
    "and you have got extended tools. So let me give you an example for each of this so that you can",
    "understand this better. A prompt is what is the capital of New Delhi. That's all the prompt is.",
    "You simply go give what is the capital of New Delhi and the LLM understands it and gives you a back",
    "understanding. Just gives it back. Now short-term memory is when you have conversational history or",
    "something in the LLM. That is what we call as ICL in context learning. So whatever you stuff",
    "inside the context window, the LLM can take it. That is your short-term memory. So you give",
    "a few short examples. You give an example like for example, what is the capital of US? I guess",
    "it's Washington DC. Washington DC. And you give a bunch of examples like this so the LLM knows",
    "what is that it has to answer. This is a short-term memory. Next you have external data. Now you take",
    "data from Wikipedia and you keep it and then give it to the LLM. That is your long-term memory",
    "because short-term memory just like a computer, the RAM, it gets reset every time you reset the",
    "conversation or the session. And then tools. You let LLM use tools like calculator, internet,",
    "python terminal and all these things. And extended tools is when you expand this much beyond that.",
    "I hope now you have understanding about the five different dimensions that we have in LLMs.",
    "A prompt, a short-term memory or in-context memory, a long-term memory or external knowledge,",
    "external data or custom knowledge, tools like calculators and a python ripple and extended",
    "tools that goes a much beyond that what we do not have currently. So these are different dimensions.",
    "Now coming into what we wanted to see is chatbot. So how do you make a Q&A bot as a chatbot is very",
    "simple. Now at this point you might have already got this idea. So you take a prompt and you give",
    "it to the LLM where you can have short-term memory, in-context memory, in-context learning.",
    "For example, so what is the capital of India? So what is the capital of India? You ask and the LLM",
    "answers New Delhi. This is what happens in a simple Q&A bot. But how do you make it a conversational",
    "bot or a chatbot? By adding a new dimension called a short-term memory. And how do you do that?",
    "You keep all these things that you are conversing into the chat conversational history.",
    "So what this gives the ability for an LLM to do is when you say what is the capital of India,",
    "it says New Delhi. Then you can just simply go and say what are some famous cuisines there.",
    "So at this point the LLM would have an understanding you are talking about New Delhi because that",
    "conversation is stored there in the LLM's short-term memory or the in-context memory.",
    "So the LLM can do something called IECL in-context learning and give you the response back.",
    "And that is how you upgrade in the pyramid by building a Q&A bot, giving a new dimension",
    "called history and then making the Q&A bot a chatbot so that it can converse. Now chatbot has",
    "applications everywhere that you can turn towards. You have got chatbot in customer support. You have",
    "got chatbot on websites. You have got chatbot for education. Like you've seen a lot of demos from",
    "Khan Academy. So chatbot is quite versatile. It almost has its purpose in every single business",
    "or domain that you can think of. Now people were using chatbot, but you know chatbot itself is not",
    "enough. Why? We already know the answer to the question. Can you pause and answer if you know",
    "the answer? So why is that chatbot is not enough for a lot of use cases? The answer to the question",
    "is chatbot stops with only short-term memory. You need long-term memory or you need external",
    "memory. For example, I ask what is the capital of India? It says New Delhi. What are the famous",
    "cuisines there? It will give me an answer. Quite valid. LLM is doing its job. So let's say I'm a",
    "company. So I'm an organization. Let's take Apple for an example. Now I ask who is the CEO of Apple?",
    "Of course, the internet has information about it. So it will say Tim Cook. That's quite easy.",
    "Now if I go say who is the manager of the team handling iPhone 16? Will it answer? No. I mean,",
    "it might answer because it hallucinates a lot, but the answer would not be correct. And that has",
    "become a big bottleneck in a lot of enterprise use cases because you do not just need internet",
    "knowledge. You do not just need the knowledge that the LLM has got. You need more than that. And that",
     "is the custom knowledge component or the external knowledge component that you need, the dimension",
    "that you need to make your LLM slightly more than just a chatbot. And that is where a new technique",
    "called RAG comes into picture. Retrieval Augmented Generation, where you use the knowledge that you",
    "provide or you call it a long-term memory. You use the documents, the internet, the sources,",
    "everything that you have around. And you use that knowledge to send to route to LLM and then make",
    "the LLM use the leverage that knowledge. And now at this point, probably you might have guessed it.",
    "See, first we had only prompt one dimension. Second, we had short-term memory, two dimension.",
    "Now we have external knowledge, which is three dimensions. So this LLM is at the center of three",
    "different things. You have got prompt, you have got short-term memory, and you have got long-term",
    "memory. To make you understand this better, so I'm going to take you to the RAG. So how does the RAG",
    "look like? So you have got the LLM at the center of it. You have got your data somewhere available.",
    "So it could be on different structures. It could be on database. Most organizations have data in",
    "their database, structured database, RDBMS database. Then you have got documents, which are unstructured",
    "like PDF, HTML files, internal portals, blah, blah, blah, blah, blah. Then you have got APIs.",
    "Let's say you are a sales team. Probably your data is in some CRM or Salesforce. So you need",
    "a programmatic call to make the call and get the answer back. So your data could be of these",
    "different places. Could be like structured database, like RDBMS system. It could be unstructured",
    "documents, PDFs, HTML documents, anything that you have locally. And then you have got programmatic",
    "access. Like you're a marketing team. You need data from Google ads. You're a sales team. You",
    "need data from Salesforce. Your company is heavily into IT. So you need data from AWS,",
    "like billing costs and all other things. So this is programmatic. So you use one of these methods,",
    "a structured parsing or unstructured parsing, a programmatic call, and take all the input data",
    "and create an index. An index is what Google creates at every single moment. You have got all",
    "these websites. What Google does is Google creates this index. So it is easier for Google",
    "to go traverse when somebody is asking a question. And that's how Google became popular.",
    "Before Google, people were using totally different thing. Google came up with something",
    "called PageRank algorithm. At the fundamental of PageRank algorithm, you have got this index",
    "with the different parameters, of course. And definitely we're not building Google, but so",
    "index is what we are building. It makes it easier for you to understand what is inside the data.",
    "So now a user comes in, asks a question. What is the question? Who is the manager of iPhone 16 team?",
    "So that question goes to the index. This system, particular system, takes that and picks only the",
    "relevant information. See, this index might have information about all the teams, iPhone 16, Apple,",
    "Vision Pro, billing, accounting, procurement, marketing, blah, blah, blah, blah, blah. So it",
    "has all the information. What you are interested in is only this particular piece, which is what",
    "you asked, which is iPhone 16 manager. So this particular part is where it takes only the",
    "relevant information from the index, and then it matches with the query, the prompt that you give,",
    "and then it finally sends it to the LLM. The prompt, what you asked, and the data that you",
    "extracted, and it goes to the LLM. LLM gives the answer back to the user. This is quite different",
    "from the chat bot application, if you see. I'll give you an example why. So in the chat bot,",
    "all you are doing is you have a memory, question is there. Sometimes you might do, let's say,",
    "a long-term memory by doing user profiling. I'll ignore this for now. You don't have to use this",
    "now. Ignore this for now. So what you're doing is you have a question, you're sending it as a prompt,",
    "and you have memory that also goes to the prompt, because that's how you can do it.",
    "And you have LLM answering this question, and you get the answer back. Now you might ask me, hey,",
    "why do I need to put my thing in the external data and create an index? Rather, why can't I",
    "keep it in memory? If you have got this question at this point, that is a very important question,",
    "and you are thinking in the right direction. In fact, people who reached at this point,",
    "you can tell me whether you know the answer or not. The reason why we cannot do this or we could",
    "not have done it early in these days of LLMs is due to an important factor called CTX window.",
    "What is CTX window? CTX window is nothing but called context window. This internal memory",
    "and question or the short-term memory and the question is bounded by what is the context window",
    "of this particular LLM. So you have an LLM. The LLM might have context window like 4K,",
    "which is quite popular these days, or 8K. And even general LLMs have like 1 million as context",
    "window. So context window is there. Now what you are actually doing here is you have a question,",
    "the LLM answers. So you have a question 1, and answer 1 comes back. Then you have a question 2,",
    "then you have answer 2. By the time you go to question 3, what you are sending to the LLM is",
    "not just your question 3, you are actually sending all these things. So let's say this is 2K,",
    "this is 1K answer, then again 2K question, 1K answer, and let's say this is a 2K question.",
    "So at the end of the day, when you are hitting the third level of conversation, I am kind of",
    "exaggerating, but let's say 2 plus 1, 3, 3, 6, 8. So you have already hit 8K context window. So if",
    "you have got 8K token model, at this point your model will hit out of memory error or it cannot",
    "hold it in short-term memory. And that is exactly why you need RAG, Retrieval, Augmented Generation,",
    "because this one is not bound by the conversation. Of course, you are going to keep it in",
    "conversation, but you don't have to stuff everything inside your question. Rather,",
    "you can keep it inside your index, because you've already indexed and you can keep it.",
    "And only the bit that is relevant comes to you. And now you might be asking, how is that possible?",
    "And for that, you go into a separate tangential side that talks about semantics and semantic",
    "search and all the other things, embedding semantic search. That is quite out of scope.",
    "If you want to go deep, you should read RAG. Llama Index is an excellent library for you to read",
    "about RAG. They have got a really good developer relation system. They have got a lot of articles",
    "and you should definitely read about Llama Index and RAG if you want advanced RAG.",
    "But I hope you get the point. Going back to our system that we put together. So what do we have?",
    "We have a Q&A system at the front, which just takes an input, gives an output, nothing else.",
    "Then you have got the chatbot. The input plus history goes together. That is all short-term",
    "memory. You get the output. The output also goes back to the input. That's why you keep the",
    "conversation history. Then you have got a RAG Retrieval Augmented Generation. The reason why it",
    "is called Retrieval Augmented Generation is because you have got a retrieval component that you augment",
    "with the LLM component and then you generate the response back. So that is Retrieval Augmented",
    "Generation and the applications are enormous. There are a lot of startups in 2024 when we are",
    "recording this. A lot of startups just doing RAG. So if you can build a RAG solution today",
    "in 2024, you can probably even raise fund or you can be a good successful SaaS. There are a lot",
    "of companies making really good money, solid money out of it. I'll give you an example. In fact,",
    "one thing that I've seen, sitegpt.ai. If you go to sitegpt.ai, it says, make AI your expert",
    "customer support agent. And I know this is a product that is making a lot of money,",
    "hundreds and thousands of dollars. And at the foundation of it, it is a RAG. It takes all the",
    "information that is available in your website, indexes it, or we call it data ingestion,",
    "ingestion and indexes it. And when you ask a question, it just gives you an answer back.",
    "That's it. It's not just a normal chatbot. It is a chatbot that can answer based on the existing",
    "data. So if you are breaking into LLM today, I would strongly encourage you to do some RAG system.",
    "That is by default something that you should do. So if you are a university student watching this,",
    "if you're an early in career professional, I would say you should build a couple of RAG",
    "examples so you know, there are a lot of nuances in RAG. Like how do you improve indexing? How do",
    "you improve indexing by changing chunking? What kind of algorithms you use for embedding and",
    "what kind of models are good with RAG? Whether you put the text at the top, is it good? Whether",
    "you put the text at the bottom, is it good? If the text is in the middle, it is good.",
    "There are a lot of components to RAG. RAG is not just simply what we discuss usually on this channel.",
    "You can go advanced RAG and I would strongly encourage you to spend some time in RAG unless",
    "you want to get into something that is quite exciting and interesting. But before we do that,",
    "I would like to quickly show you one more thing that not a lot of people discuss when we talk",
    "about LLMs. It is not necessarily RAG. It is just like using short-term memory. So it doesn't use",
    "long-term memory, but it has its own potential, which is to use LLMs, large language models,",
    "for classical NLP tasks, classical NLP downstream tasks. For example, let's say you want to build",
    "a text classification system. What is a text classification system? You give a sentence,",
    "for example, the movie was complete crap. Now, is it positive or negative? Positive or negative?",
    "You choose, you build, you train a text classification model just to figure out this.",
    "For example, or the other example I can give is you have a review. Let's say the movie was amazing",
    "and the actress was exceptional. Now you try to build a model that will say what kind of review",
    "is this? For example, is this review about movie theater or director or actor? So now you know this",
    "is an actor based. So this is what a text classification in classical NLP is. There are",
    "a lot of other tasks that you do in classical NLP. What you can do is without having to build",
    "your custom model, like let's say BERT-based model, XGBoost-based models, you can use LLMs,",
    "large language models, for classical NLP problems because large language models are really good in",
    "context learning. And with the current memory that you have got, with a few short examples or",
    "tree of thoughts or a chain of thoughts, you can make your large language models a good zero short",
    "NLP classifier, or it is applicable for a lot of other tasks as well. So one thing that not a lot",
    "of people are exploring, I would encourage you to explore if you work in classical NLP problems,",
    "like labeling or a text classification, entity recognition, whatever it is, you can leverage",
    "LLM. Now the question is, do you want an LLM based solution? That's a different topic. I'm",
    "not talking about you looking for a nail because you have got a hammer. I'm just saying that this",
    "is a good opportunity. Wherever you don't want to build models, you can use this. But of course,",
    "if you can build models, that will be probably cheaper than making calls to LLMs and getting",
    "answer back. But summarization, text classification, entity recognition, I think LLMs are exceptional,",
    "zero short LLM for downstream tasks, and you should definitely leverage it. Now with this,",
    "we have arrived at RAG. So we have arrived at RAG and we already know what is RAG.",
    "Now we are entering into a very interesting phase about what everybody is obsessed with,",
    "what everybody's love, agents. Very recent announcements from Google, Microsoft,",
    "previously OpenAI. In every announcement, you would have seen two important things as a common",
    "trend. One is you would have seen multimodality, multimodality. What does it mean? It just simply",
    "means instead of just chatting with the text, you can chat with images, you can ask questions in",
    "voice, it can respond back in speech, you can send videos. So one important trend that you're seeing",
    "is multimodality. And the second important trend that you see everywhere is agents.",
    "Multi-agent setup, where you have got multiple agents, you can summon these agents to do certain",
    "tasks and these agents will do it for you, just like Men in Black, MIB. They have a purpose and",
    "they will do certain tasks. But before I jump into agents, I want to actually introduce you to",
    "another important concept called function calling. Function calling is the precursor",
    "to LLM agents. In function calling, what you do is you have a short-term prompt,",
    "you have a prompt, you have short-term memory. Sometimes you need external memory, sometimes",
    "you don't need external memory. But you are giving the ability of calling external tools.",
    "And you are giving the ability of calling external tools by doing something called function",
    "calling. Function calling, to be honest, is a terrible name because you're not calling",
    "any function here. You're not making the LLMs call anything, not at all. All you're doing is",
    "you're forcing the LLM to give a structured response back so you can call an LLM. I'll give",
    "you an example of what is function calling. So let's say that you have a weather API.",
    "I think everybody goes with weather API, so I'm going to skip. Let's say you have a currency",
    "converter. What kind of things a currency converter needs? You need input currency,",
    "you need output currency, you need date, you need amount. Technically, these are the four things you",
    "need. What is the amount that you want to convert for? What is the input currency? What is the output",
    "date for which you want to do currency conversion? Let's keep as a simple API. Now, typically,",
    "when you go to an LLM and say, what is USD to INR today? First of all, LLM may not understand",
    "what is today. LLM might know USD, LLM might know INR, but LLM's memory is frozen because it's a",
    "snapshot. A large language model is a snapshot. So its memory has been frozen to, let's say,",
    "September 2023 or something like that. So what it cannot do is it cannot give you the latest",
    "information. And you cannot do this with, I mean, you can do this with RAG, kind of like you can",
    "every day take knowledge, ingest, keep it in your memory, and then give it back. Not very efficient.",
    "Expand this to stock market, but daily data doesn't matter because everything changes like every",
    "minute and every second. So you need something instant. What do you do? You call an API. If you",
    "are a programmer, that's what you would naturally do. You call an API. Now, if you want to call an",
    "API, what you need to call an API. So let's say this is the information. What I need at the end",
    "of the day, I want to call it currency converter. And I'll say input, output, date, amount. I need",
    "to make a call like this. So I need four arguments that is solid. Input could not be like, oh,",
    "United States dollar. And some other time I'll be like USD. Some other time I'll be like US dollar.",
    "I mean, that will not work, right? You need a specific format for everything. Your, let's say,",
    "amount should be an integer, right? This should be a date object. So you need to force this LLM to",
    "give you a particular response back. Otherwise, what happens is this LLM will throw you anything.",
    "For example, what I want to say is, what is USD and INR? So you'll be like, oh, USD and INR,",
    "so on September, 2023. So you have to force guide the LLM to make a particular type of output.",
    "And somehow universally, everybody has accepted that format is going to be JSON,",
    "except Anthropic, which absolutely loves XML. So if you use Anthropic, you use XML. If you",
    "use any other model, you use JSON. So you're forcing an LLM to give you a structured response",
    "back, a JSON that can help you make this function call. You can call this function with the JSON.",
    "So a guided response into a JSON is what everybody calls function calling. You don't",
    "necessarily call the function in function calling, but you get the output that will",
    "help you call function call, right? Clear? Now that is exactly what is a precursor to",
    "agent, because in a function call, you have the ability to call a function and agents are nothing",
    "but a bunch of function calls stitched with tools. So what do we have in agents? We have a bunch of",
    "function calls plus tools. And I would like to introduce to you a very interesting solution",
    "that can help you understand more about agents. If you are too old in the AI world, you would",
    "have probably recognized this immediately. And this was the workflow of something called",
    "baby AGI. So baby AGI was quite a popular thing back in the day. I mean, back in the days like",
    "less than one year before, I guess, or maybe more than one year. A function call is what I said is",
    "the foundation of agents. But what is an agent? Now, if you have seen our pyramid, you would know",
    "our agent sits right at the top, like closer to what we are aspirational figures. Now,",
    "what is this agent? How do you define an agent? So it's simple. First of all, a chat bot drag,",
    "all of these guys, if you see here, they end a text or, you know, some kind of thing like input,",
    "output, images, video, all these things, right? That's where they end. One of these modalities,",
    "they're done. What you achieve with agent is something that is absolutely stunning.",
    "You don't stop at text response. You stop at an action. You trigger an action. And that is what",
    "agents are simply. You take LLM, you connect them with tool, you give them a purpose or goal.",
    "That is your agent. And that is exactly what baby AGI has done back in the day. Like there are",
    "multiple agents now, but if you see baby AGI, which is a very wonderful framework, you can see",
    "that there is a task, like there is something that has to happen. There are certain tools,",
    "like for example, vector DB and all the other things are there. And every agent has a purpose,",
    "like, okay, you have to execute, you have to return, you have to do something, you have to",
    "do something and they have a goal. So you have tools, purpose slash goals and LLMs and this",
    "all together work for a common goal. And that is your agent. There are multiple agent frameworks",
    "that are quite popular these days, the crew AI, land graph, you have got a pie autogen. And most",
    "of these things you will see. First, you have to define a role. You have to define a goal, define",
    "a goal, a role goal. And then you have to say which LLM that you want to use as a backend engine.",
    "And then you put together a system of one, this is single agent. Now you put together like this",
    "as a team, that is your multi agent setup. With agents, people are doing amazing things.",
    "You can make an agent book your ticket. You can make an agent, let's say read something,",
    "distill something, create an order, publish the blog post. You can summon these agents to do a",
    "lot of things. And personally for me, the most time that I spent reading about agents, because",
    "it's becoming quite obvious that agents are the next frontier in the way we can take LLMs forward.",
    "I mean, there are a lot of different things, but at least personally, I'm quite interested",
    "in automation usually. And I think agents are going to be the next big thing in, I mean,",
    "currently itself is a big thing. Google has got Google's own projects, like they call",
    "their own agents. I don't know what they call, they have a lot of different names.",
    "Open AI has its own agents. And every time you talk to some company, you speak about agents,",
    "because you want to summon these agents. You want to connect these LLMs to like different",
    "dimension and want this dimension that what we are connecting is the tools dimension. So",
    "you take LLMs, you have the function calling ability. And once you connect them to tools,",
    "you are unlocking the potential of something immense. And that is what you call as agents.",
    "I'm not going deep into agents because this is probably, I'm hoping it to be a series,",
    "depending upon how you all like it. But in the series, my next focus is going to be agents.",
    "So agent is quite closer to the top. And that takes us to the almost the end of the video,",
    "which is what is our aspirational thing? What is that we are all trying to go towards,",
    "which is LLMOS. And this is inspired by Andhri Karpathy, who created this amazing structure.",
    "So what is happening here? This talks about using LLM at the center of a conversation,",
    "center of an operating system. If you go back in the day, computer was created just for simple",
    "calculation purpose. You want to add A and B, you want to keep A for one and B for two.",
    "And then you want to add, that's what like initially computer was started like very,",
    "very, very, very back in the day. And computation started increasing,",
    "computation started becoming less expensive, more compute. Then we have the computer that we have",
    "today. Andhri Karpathy is arguing, can we have a similar vision for LLM? And where the vision is",
    "you keep LLM at the center, right? You keep LLM at the center. And at the center with LLM,",
    "you have RAM, which is the short-term memory or the context window. Then you have long-term memory,",
    "the disk system that can be used with RAG. Then you have the agent structure that you have with",
    "tools. And then you connect it with internet. And when you connect it with other LLMs to have",
    "like a multi-agent setup or like a peripheral setup. And then you have your peripheral devices",
    "where you have got audio and video. Can we put together a system with all these things working",
    "towards a common goal? And that will ideally become your large language model operating system.",
    "This is quite a vision at this point. There are certain implementations available at this point.",
    "Those implementations are based on current understanding. They are mostly,",
    "let's say, LLMs plus function calling plus agents, multi-agent more tools. That is what",
    "the current LLM OS is. It's not like radically has a different total view altogether. And that's why",
    "if you see even in my framework that I've created, LLM OS is currently developing and it is everything",
    "that we have got. The tools, the extended tools, the peripheral tools, the long-term memory with",
    "short-term memory, just one input from the user where it can run itself and then it can execute",
    "certain things. I think that is the future that we are heading. I'm not sure when we are going to do",
    "it, but if somebody says something AGA for me today, AGA could be like this could be like the",
    "baby AGA. I mean, I don't trust AGA as a concept anytime soon, but yeah, leaving the",
    "conscious thing, consciousness and all the other things out. I would say LLM OS is at the top where",
    "we can expect something closer to AGA happen and all these things lead us up to there. So I wanted",
    "to keep this video brief, but this video is already going to be like more than half an hour. I wanted",
    "this to be like a crash course where you understand if you don't know anything about LLM OS, maybe",
    "you have not taken any course. So this is going to help you to see how the future of LLM OS is",
    "coming and what led us up to there. And let me know in the comment section, if you like this kind",
    "of content, I'll put together more. This took me a lot of time to create the framework design,",
    "put it in a particular thought process to make it make it understandable. And this is basically",
    "what a lot of LLM courses offer. So I'm definitely looking forward to hear more feedback. And if you",
    "like this kind of format, subscribe to the channel. See you in another video. Happy Prompting."
]
    


In [7]:
lista2 = [
    "let's talk about tech stack. I get a lot of suggestion from software engineers who",
    "mentioned that I could save some money by switching to another tech stack or",
    "maybe a new UI components library that's trendy but I never listened to them and I",
    "always stick to the exact same tech stack and thanks to that in the last two",
    "years I was able to ship 21 products and those products are getting 260,000",
    "page views per month and are making over 50,000 US dollars in the revenue and I'm",
    "working solo and I handle the customer support and there is barely any problem",
    "and the tech stack only cost 67 US dollar per month. Alright now let me show you",
    "exactly the tech that I use. Alright so for the front-end part I use React.js",
    "it's a very popular JavaScript library it has a bunch of extra library like a",
    "drag-and-drop features for instance that can plug and play right away and some",
    "people say that it's quite complicated I only know 10% of React I just use",
    "use state and use effects in 99% of the cases it's way enough for all my apps",
    "then I pack all that into a Next.js application it's very good for page",
    "routing and to have nice pages ranking on Google and it's also super easy to",
    "use. To style my apps I use Tailwind CSS I feel like it's much easier than CSS",
    "because the style is written directly on the right HTML tags so it's easy for me",
    "to understand what's going on and on top of that I use a UI library called daisy UI",
    "it has a bunch of components like buttons, form, cards that make things way",
    "faster to ship there are a bunch of new design trends popping up every day which",
    "means new UI components library is coming out and to me that's just",
    "distraction customers don't care if I'm using the latest gradient colors or",
    "whatever so I stick to that UI library it's clear it's simple they have 30 or",
    "20 plus themes that I can customize so whenever I ship a new app I would pick a",
    "theme and find a cool typeface on Google Fonts and boom I have an entirely",
    "design cool-looking apps that's up and running all right and for the back end",
    "part for my API I would use Next.js serverless functions whenever I clone a",
    "Next.js project I just add a file in the API folder and I have an entire API",
    "endpoint that's up and running I don't need to install packages I don't need to",
    "maintain an OS it's just working so well for me and I never had any trouble with",
    "overcharge for serverless so all good with that for the database I use MongoDB",
    "I don't know why developers don't like MongoDB I love it it's so simple I click",
    "a button I open a new database and in minutes I can make queries and I can add",
    "anything I want to that database it's just so simple and I absolutely love it",
    "I set up a M0 cluster for each new projects that I start which is free and",
    "just in case things go wrong I do occasional backups cost me about $10 per",
    "month and so if something goes wrong I can still use a database backup and",
    "restore all the previous data when I'm cutting on my computer I use a local",
    "version of MongoDB that lets me avoid making a network request to the",
    "production database when I'm cutting so I can code in a train in a plane remote",
    "somewhere in the countryside if there is no internet and the database is here and",
    "I can do everything I want and I also don't mess up the production database",
    "just in case on top of MongoDB I add mongoose which simplifies the queries",
    "they're more English friendly I would say all right and all my apps use user",
    "logins to send things like magic links so that they can sign up with a secure",
    "token I would use mailgun as an email provider for that it's free to get",
    "started it's super easy and you can set up a bunch of domains that you want for",
    "each of the apps at the moment I'm recording the video I spend three to",
    "four US dollar per month in order to send three to four thousand emails which",
    "I think is really cheap and finally I use Stripe in order to process payments",
    "whether it's one-time payments subscriptions credits it works for",
    "literally anything Stripe API is a little bit daunting when you are getting",
    "started there are a bunch of things going on so I will link to an article",
    "that I wrote to help you handle Stripe subscriptions all right so for the",
    "hosting part where my websites are actually leaving I use Versal for that",
    "so I push my changes to a GitHub repo and then Versal automatically pulls the",
    "changes that I made and deploys it to Versal instance and I have 21 projects",
    "running on Versal at the moment it gets over 50,000 visitors a month and I still",
    "pay $20 a month so overall pretty good really happy with Versal and the",
    "simplicity it's bringing and all my databases are hosted on MongoDB Atlas",
    "where I have over 21 clusters and most of them are M0 cluster which are free",
    "to monitor data to actually see what's going on in my apps I use a few services",
    "first is plausible it's a simple script analytics that lets me track how many",
    "visitors are my site and sometimes occasionally I will track events like",
    "the people click on that button or not it's costing me $29 per month at the",
    "moment which is surprisingly the most expensive part of my tech stack if I",
    "receive a customer email support and something goes wrong there's a bug that",
    "I don't know where it's coming from I will check the virtual deployment logs",
    "it's free to check the last 24 hours and it's usually when I catch the bug and",
    "for advanced monitoring things like split testing I would do it manually",
    "with a database I would store some kind of like integer in the database",
    "randomly and then just make the relevant query in the database see I don't let's",
    "say like the last hundred signed up customers filter them by their split",
    "test number and see which one purchased which one didn't I don't use any other",
    "services for that because I barely track any data and if I need to I would just",
    "use the raw MongoDB queries and finally some of my products need specific",
    "features so for instance I would use AI models for products like logo fast where",
    "I help people generate logos or for workbook PDF to help the help again",
    "people will generate workbook to learn a new language I use GPT-4 under the hood",
    "and I spend about $20 per month in open AI credit if I need any other cloud",
    "services I would go to Amazon Web Services the two most common that I use",
    "are AWS S3 a big bucket where I can host images this is what I use on indie page",
    "where people upload pictures of their startups or their profile picture this",
    "is where the image would go and I also use from time to time AWS in cloud front",
    "it's a CDN so images or videos load faster for things like a course for",
    "instance and a final touch it is not really part of a tech stack but I hire",
    "a junior software engineer called github copilot it cost me $10 per month and",
    "it's helping me code way faster I'll link to another video I made about how I",
    "turn on a startup ID into the first $1,000 in four days using github copilot",
    "when I learned how to code I remember struggling finding the perfect tech stack",
    "but I wish I knew this tech stack don't matter customers don't care about the",
    "visible what matters is that you pick one tech stack and you stick to it it's",
    "like a muscle the more you practice one rep the more comfortable you'll get with",
    "it and the faster you'll be able to do it and the faster you ship the faster",
    "you'll get customers alright that's the end of the video as always if you like",
    "the video you can like the video if you loved it you can subscribe to the",
    "channel please also let me know in the comments if there are any topics you",
    "would like me to cover and until the next video just ship it cheers by the",
    "way I got someone to help me edit those videos so I can spend more time",
    "building apps please let me know what you think about the editing"
]


In [112]:
listas3 = [
    {"id": 1, "start": 0.0, "end": 4.0, "text": "No sé de dónde viene, revisaré los registros de implementación virtual"},
    {"id": 2, "start": 4.0, "end": 8.0, "text": "es gratis revisar las últimas veinticuatro horas y es usualmente cuando detecto el error y"},
    {"id": 3, "start": 8.0, "end": 12.0, "text": "para monitoreos avanzados como pruebas A B, lo haría manualmente"},
    {"id": 4, "start": 12.0, "end": 16.0, "text": "con una base de datos almacenaría algún tipo de número entero en la base de datos"},
    {"id": 5, "start": 16.0, "end": 20.0, "text": "aleatoriamente y luego simplemente haría la consulta relevante en la base de datos, no sé, digamos"},
    {"id": 6, "start": 20.0, "end": 24.0, "text": "como los últimos cien clientes registrados, filtrarlos por su número de prueba"},
    {"id": 7, "start": 24.0, "end": 28.0, "text": "y ver cuál compró y cuál no. No uso ningún otro"},
    {"id": 8, "start": 28.0, "end": 32.0, "text": "servicio para eso porque apenas rastreo datos y si necesito hacerlo, simplemente"},
    {"id": 9, "start": 32.0, "end": 36.0, "text": "uso las consultas crudas de Mongo debe y finalmente algunos de mis productos necesitan"},
    {"id": 10, "start": 36.0, "end": 40.0, "text": "características específicas, por ejemplo, usaría modelos de I A para productos como logo rápido donde"},
    {"id": 11, "start": 40.0, "end": 44.0, "text": "ayudo a la gente a generar logotipos o para PDF de trabajo para ayudar a la gente nuevamente"},
    {"id": 12, "start": 44.0, "end": 48.0, "text": "a generar libros de trabajo para aprender un nuevo idioma. Uso G P T cuatro como base"},
    {"id": 13, "start": 48.0, "end": 52.0, "text": "y gasto alrededor de veinte dólares por mes en crédito de Open A I. Si necesito cualquier otro servicio en la nube"},
    {"id": 14, "start": 52.0, "end": 56.0, "text": "iría a Amazon Web Services, los dos más comunes que uso"},
    {"id": 15, "start": 56.0, "end": 60.0, "text": "son A W S S tres, un gran depósito donde puedo alojar imágenes, esto es lo que uso en la página indie"},
    {"id": 16, "start": 60.0, "end": 64.0, "text": "donde la gente sube fotos de sus startups o su foto de perfil, aquí es donde irían las imágenes, y también uso de vez en cuando A W S Cloud Front"},
    {"id": 17, "start": 64.0, "end": 68.0, "text": "es un C D N, así que las imágenes o videos cargan más rápido para cosas como un curso, por"},
    {"id": 18, "start": 68.0, "end": 72.0, "text": "ejemplo. Y un toque final, no es realmente parte de una pila tecnológica, pero contrato"},
    {"id": 19, "start": 72.0, "end": 76.0, "text": "a un ingeniero de software junior llamado GitHub Copilot, me cuesta diez dólares por mes y"},
    {"id": 20, "start": 76.0, "end": 80.0, "text": "me ayuda a codificar mucho más rápido. Enlazaré otro video que hice sobre cómo"},
    {"id": 21, "start": 80.0, "end": 84.0, "text": "convertí una idea de startup en los primeros mil dólares en cuatro días usando Gitjub Copilot"},
    {"id": 22, "start": 84.0, "end": 88.0, "text": "cuando aprendí a programar recuerdo que me costaba encontrar la pila tecnológica perfecta"},
    {"id": 23, "start": 88.0, "end": 92.0, "text": "pero ojalá hubiera sabido que la pila tecnológica no importa, a los clientes no les importa lo"},
    {"id": 24, "start": 92.0, "end": 96.0, "text": "invisible, lo que importa es que elijas una pila tecnológica y te mantengas con ella, es"},
    {"id": 25, "start": 96.0, "end": 100.0, "text": "como un músculo, cuanto más practicas una repetición, más cómodo te sentirás con"},
    {"id": 26, "start": 100.0, "end": 104.0, "text": "ella y más rápido podrás hacerlo, y cuanto más rápido envíes, más rápido"},
    {"id": 27, "start": 104.0, "end": 108.0, "text": "conseguirás clientes. Bueno, ese es el final del video, como siempre, si te gustó"},
    {"id": 28, "start": 108.0, "end": 112.0, "text": "el video, puedes darle like, si te encantó, puedes suscribirte al"},
    {"id": 29, "start": 112.0, "end": 116.0, "text": "canal. Por favor, también házmelo saber en los comentarios si hay algún tema que"},
    {"id": 30, "start": 116.0, "end": 120.0, "text": "quieres que cubra y hasta el próximo video, simplemente envíalo, saludos. Por cierto, conseguí a alguien que me ayude a editar esos videos para poder pasar más tiempo"},
    {"id": 31, "start": 120.0, "end": 124.0, "text": "construyendo aplicaciones. Por favor, déjame saber qué piensas sobre la edición"}
]

In [113]:
lista_200 = listas3[10:20]
lista_200

[{'id': 11,
  'start': 40.0,
  'end': 44.0,
  'text': 'ayudo a la gente a generar logotipos o para PDF de trabajo para ayudar a la gente nuevamente'},
 {'id': 12,
  'start': 44.0,
  'end': 48.0,
  'text': 'a generar libros de trabajo para aprender un nuevo idioma. Uso G P T cuatro como base'},
 {'id': 13,
  'start': 48.0,
  'end': 52.0,
  'text': 'y gasto alrededor de veinte dólares por mes en crédito de Open A I. Si necesito cualquier otro servicio en la nube'},
 {'id': 14,
  'start': 52.0,
  'end': 56.0,
  'text': 'iría a Amazon Web Services, los dos más comunes que uso'},
 {'id': 15,
  'start': 56.0,
  'end': 60.0,
  'text': 'son A W S S tres, un gran depósito donde puedo alojar imágenes, esto es lo que uso en la página indie'},
 {'id': 16,
  'start': 60.0,
  'end': 64.0,
  'text': 'donde la gente sube fotos de sus startups o su foto de perfil, aquí es donde irían las imágenes, y también uso de vez en cuando A W S Cloud Front'},
 {'id': 17,
  'start': 64.0,
  'end': 68.0,
  'text': 

In [114]:
len(lista_200)

10

In [115]:
cadena_200 = json.dumps(lista_200, ensure_ascii=True)
cadena_200

'[{"id": 11, "start": 40.0, "end": 44.0, "text": "ayudo a la gente a generar logotipos o para PDF de trabajo para ayudar a la gente nuevamente"}, {"id": 12, "start": 44.0, "end": 48.0, "text": "a generar libros de trabajo para aprender un nuevo idioma. Uso G P T cuatro como base"}, {"id": 13, "start": 48.0, "end": 52.0, "text": "y gasto alrededor de veinte d\\u00f3lares por mes en cr\\u00e9dito de Open A I. Si necesito cualquier otro servicio en la nube"}, {"id": 14, "start": 52.0, "end": 56.0, "text": "ir\\u00eda a Amazon Web Services, los dos m\\u00e1s comunes que uso"}, {"id": 15, "start": 56.0, "end": 60.0, "text": "son A W S S tres, un gran dep\\u00f3sito donde puedo alojar im\\u00e1genes, esto es lo que uso en la p\\u00e1gina indie"}, {"id": 16, "start": 60.0, "end": 64.0, "text": "donde la gente sube fotos de sus startups o su foto de perfil, aqu\\u00ed es donde ir\\u00edan las im\\u00e1genes, y tambi\\u00e9n uso de vez en cuando A W S Cloud Front"}, {"id": 17, "start": 64.0, "e

In [116]:
lista_traduccion = []

In [118]:
template = """
Eres un experto traductor del inglés al español y un excelente programador.

# TAREA:
Tu objetivo es tomar el JSON que te dan, que es una lista de las transcripciones de un video con el id, start, end y el text, y traducir el text al español :

1. Mantén el formato de JSON, sustituyendo únicamente las cadenas de cada text por la traducción al español.
3. Traduce el texto al español, pero conserva las marcas o terminologías académicas como clases de programación en inglés.
4. Asegúrate de que las frases sean coherentes. Si no tienen sentido al traducirlas literalmente, ajústalas para que mantengan el contexto.
5. Mantén la misma cantidad de items en la representación de la lista.
7. Entregarás tu respuesta en fomato JSON donde la key se llama 'traduccion' y el valor sea la lista de strings.

# CONTEXTO:
Estás trabajando para una empresa que transcribe vídeos del inglés al español. A tí te envian un JSON de python para que tu traduzcas cada parametro text al español. Tu tarea es encargarte de traducir cada text y de entregar un JSON respetando el formato con el que te lo entregan donde entregarás un JSON con una key 'traduccion' y como valor, la lista que te pasaron traducida más sus parametros. Es muy importante que repetes el formato .

# Ejemplo 1
input:
[
    {{"id": 1, "start": 0.0, "end": 4.0, "text": "people will generate workbook to learn a new language I use GPT-4 under the hood"}},
    {{"id": 2, "start": 4.0, "end": 8.0, "text": "and I spend about $20 per month in open AI credit if I need any other cloud"}},
    {{"id": 3, "start": 8.0, "end": 12.0, "text": "services I would go to Amazon Web Services the two most common that I use"}},
    {{"id": 4, "start": 12.0, "end": 16.0, "text": "are AWS S3 a big bucket where I can host images this is what I use on indie page"}},
    {{"id": 5, "start": 16.0, "end": 20.0, "text": "where people upload pictures of their startups or their profile picture this"}},
    {{"id": 6, "start": 20.0, "end": 24.0, "text": "is where the image would go and I also use from time to time AWS in cloud front"}},
    {{"id": 7, "start": 24.0, "end": 28.0, "text": "it's a CDN so images or videos load faster for things like a course for"}},
    {{"id": 8, "start": 28.0, "end": 32.0, "text": "instance and a final touch it is not really part of a tech stack but I hire"}},
    {{"id": 9, "start": 32.0, "end": 36.0, "text": "a junior software engineer called github copilot it cost me $10 per month and"}},
    {{"id": 10, "start": 36.0, "end": 40.0, "text": "it's helping me code way faster I'll link to another video I made about how I"}}
]

output:
{{
    "traduccion": [
        {{"id": 1, "start": 0.0, "end": 4.0, "text": "la gente generará un libro de trabajo para aprender un nuevo idioma yo uso GPT-4 bajo el capó"}},
        {{"id": 2, "start": 4.0, "end": 8.0, "text": "y gasto unos 20 dólares al mes en crédito abierto AI si necesito alguna otra nube"}},
        {{"id": 3, "start": 8.0, "end": 12.0, "text": "servicios que iría a Amazon Web Services los dos más comunes que yo uso"}},
        {{"id": 4, "start": 12.0, "end": 16.0, "text": "son AWS S3 un gran cubo donde puedo alojar imágenes esto es lo que uso en la página indie"}},
        {{"id": 5, "start": 16.0, "end": 20.0, "text": "donde la gente sube fotos de sus startups o su foto de perfil esto"}},
        {{"id": 6, "start": 20.0, "end": 24.0, "text": "es donde la imagen iría y también uso de vez en cuando AWS en cloud front"}},
        {{"id": 7, "start": 24.0, "end": 28.0, "text": "es un CDN para que las imágenes o vídeos carguen más rápido para cosas como un curso para"}},
        {{"id": 8, "start": 28.0, "end": 32.0, "text": "instancia y un toque final no es realmente parte de una pila de tecnología, pero yo contrato"}},
        {{"id": 9, "start": 32.0, "end": 36.0, "text": "un ingeniero de software junior llamado github copilot que me cuesta $10 por mes y"}},
        {{"id": 10, "start": 36.0, "end": 40.0, "text": "me está ayudando a codificar mucho más rápido voy a enlazar a otro video que hice sobre cómo"}}
    ]
}}

# Ejemplo 3
input:
[
    {{"id": 21, "start": 80.0, "end": 84.0, "text": "let's talk about tech stack. I get a lot of suggestion from software engineers who"}},
    {{"id": 22, "start": 84.0, "end": 88.0, "text": "mentioned that I could save some money by switching to another tech stack or"}},
    {{"id": 23, "start": 88.0, "end": 92.0, "text": "maybe a new UI components library that's trendy but I never listened to them and I"}},
    {{"id": 24, "start": 92.0, "end": 96.0, "text": "always stick to the exact same tech stack and thanks to that in the last two"}},
    {{"id": 25, "start": 96.0, "end": 100.0, "text": "years I was able to ship 21 products and those products are getting 260,000"}},
    {{"id": 26, "start": 100.0, "end": 104.0, "text": "page views per month and are making over 50,000 US dollars in the revenue and I'm"}},
    {{"id": 27, "start": 104.0, "end": 108.0, "text": "working solo and I handle the customer support and there is barely any problem"}},
    {{"id": 28, "start": 108.0, "end": 112.0, "text": "and the tech stack only cost 67 US dollar per month. Alright now let me show you"}},
    {{"id": 29, "start": 112.0, "end": 116.0, "text": "exactly the tech that I use. Alright so for the front-end part I use React.js"}},
    {{"id": 30, "start": 116.0, "end": 120.0, "text": "it's a very popular JavaScript library it has a bunch of extra library like a"}}
]


output:
{{
    "traduccion": [
        {{"id": 21, "start": 80.0, "end": 84.0, "text": "hablemos de la pila tecnológica. Recibo muchas sugerencias de ingenieros de software que"}},
        {{"id": 22, "start": 84.0, "end": 88.0, "text": "me mencionaron que podría ahorrar dinero cambiando a otra pila tecnológica o"}},
        {{"id": 23, "start": 88.0, "end": 92.0, "text": "tal vez una nueva biblioteca de componentes UI que es trendy, pero nunca las he seguido y yo"}},
        {{"id": 24, "start": 92.0, "end": 96.0, "text": "siempre me mantengo en la misma pila tecnológica y gracias a eso en los últimos dos años"}},
        {{"id": 25, "start": 96.0, "end": 100.0, "text": "he podido lanzar 21 productos y esos productos están recibiendo 260,000"}},
        {{"id": 26, "start": 100.0, "end": 104.0, "text": "vistas por mes y están generando más de 50,000 dólares EE. UU. En ingresos y estoy yo"}},
        {{"id": 27, "start": 104.0, "end": 108.0, "text": "trabajando solo y manejo el soporte al cliente y casi no hay problemas"}},
        {{"id": 28, "start": 108.0, "end": 112.0, "text": "y la pila tecnológica cuesta 67 dólares EE. UU. Al mes. Bueno, ahora les mostraré"}},
        {{"id": 29, "start": 112.0, "end": 116.0, "text": "exactamente la tecnología que uso. Está bien, para la parte del front-end uso React.js"}},
        {{"id": 30, "start": 116.0, "end": 120.0, "text": "es una biblioteca de JavaScript muy popular tiene muchas otras librerías como una"}}
    ]
}}

# NOTA:
- Contesta solo con la respuesta en forma de JSON. No respondas nada que no sea JSON.
- Asegúrate de mantener la estructura de las listas con el valor de la key, entregando solo la lista con los ítems traducidos {{traduccion:[{{item1}}, {{item2}}, {{item3}}, etc.]}}.
- Por favor, proporciona una respuesta en formato de JSON. El JSON debe contener una lista. La lista debe tener exactamente el mismo número de elementos que la lista original que te he dado. No agregues ni elimines ningún elemento. Los elementos tiene que tener exactamente los mismos parametros que el original solo debes cambiar el text con la traduccion.
- Es muy importante que detectes correctamente cada elemento de la lista, el buen funcionamiento de la empresa depende de ello.
- Antes de da tu respuesta compara los ids de los items de la lista original con los de la nueva y asegurate de que esten todos , si falta alguno arreglalo antes de responder.
- Asegurate en la lista del JSON de la respuesta estan los 10 dicts como en el original.

# JSON A TRANSFORMAR:
{cadena_200}

# Respuesta:
{{"traduccion": [{{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}},{{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}]}}

"""

In [119]:
prompt_template = PromptTemplate(template=template, input_variables=["cadena_200"])


In [120]:
chain = prompt_template | llm 

In [121]:
respuesta = await chain.ainvoke(input={"cadena_200": cadena_200})
respuesta

AIMessage(content='{\n"traduccion": [\n{"id": 11, "start": 40.0, "end": 44.0, "text": "ayudo a la gente a generar logotipos o para PDF de trabajo para ayudar a la gente nuevamente"},\n{"id": 12, "start": 44.0, "end": 48.0, "text": "a generar libros de trabajo para aprender un nuevo idioma. Uso GPT-4 como base"},\n{"id": 13, "start": 48.0, "end": 52.0, "text": "y gasto alrededor de veinte dólares por mes en crédito de Open AI. Si necesito cualquier otro servicio en la nube"},\n{"id": 14, "start": 52.0, "end": 56.0, "text": "iría a Amazon Web Services, los dos más comunes que uso"},\n{"id": 15, "start": 56.0, "end": 60.0, "text": "son AWS S3, un gran depósito donde puedo alojar imágenes, esto es lo que uso en la página indie"},\n{"id": 16, "start": 60.0, "end": 64.0, "text": "donde la gente sube fotos de sus startups o su foto de perfil, aquí es donde irían las imágenes, y también uso de vez en cuando AWS Cloud Front"},\n{"id": 17, "start": 64.0, "end": 68.0, "text": "es un CDN, así que 

In [122]:
mensaje = respuesta.content
mensaje

'{\n"traduccion": [\n{"id": 11, "start": 40.0, "end": 44.0, "text": "ayudo a la gente a generar logotipos o para PDF de trabajo para ayudar a la gente nuevamente"},\n{"id": 12, "start": 44.0, "end": 48.0, "text": "a generar libros de trabajo para aprender un nuevo idioma. Uso GPT-4 como base"},\n{"id": 13, "start": 48.0, "end": 52.0, "text": "y gasto alrededor de veinte dólares por mes en crédito de Open AI. Si necesito cualquier otro servicio en la nube"},\n{"id": 14, "start": 52.0, "end": 56.0, "text": "iría a Amazon Web Services, los dos más comunes que uso"},\n{"id": 15, "start": 56.0, "end": 60.0, "text": "son AWS S3, un gran depósito donde puedo alojar imágenes, esto es lo que uso en la página indie"},\n{"id": 16, "start": 60.0, "end": 64.0, "text": "donde la gente sube fotos de sus startups o su foto de perfil, aquí es donde irían las imágenes, y también uso de vez en cuando AWS Cloud Front"},\n{"id": 17, "start": 64.0, "end": 68.0, "text": "es un CDN, así que las imágenes o vid

In [123]:
lista_string = mensaje[mensaje.find('['):mensaje.rfind(']')+1]
type(lista_string)

str

In [124]:
import ast

In [125]:
lista_ciclica = ast.literal_eval(lista_string)

In [126]:

lista_ciclica

[{'id': 11,
  'start': 40.0,
  'end': 44.0,
  'text': 'ayudo a la gente a generar logotipos o para PDF de trabajo para ayudar a la gente nuevamente'},
 {'id': 12,
  'start': 44.0,
  'end': 48.0,
  'text': 'a generar libros de trabajo para aprender un nuevo idioma. Uso GPT-4 como base'},
 {'id': 13,
  'start': 48.0,
  'end': 52.0,
  'text': 'y gasto alrededor de veinte dólares por mes en crédito de Open AI. Si necesito cualquier otro servicio en la nube'},
 {'id': 14,
  'start': 52.0,
  'end': 56.0,
  'text': 'iría a Amazon Web Services, los dos más comunes que uso'},
 {'id': 15,
  'start': 56.0,
  'end': 60.0,
  'text': 'son AWS S3, un gran depósito donde puedo alojar imágenes, esto es lo que uso en la página indie'},
 {'id': 16,
  'start': 60.0,
  'end': 64.0,
  'text': 'donde la gente sube fotos de sus startups o su foto de perfil, aquí es donde irían las imágenes, y también uso de vez en cuando AWS Cloud Front'},
 {'id': 17,
  'start': 64.0,
  'end': 68.0,
  'text': 'es un CDN, así 

In [127]:
len(lista_ciclica)

10

In [128]:
from TTS.api import TTS
import torch
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
import numpy as np
import pyaudio

In [129]:
xtts_configuracion = XttsConfig()
xtts_configuracion.load_json("XTTS-v2/config.json")
xtts_modelo = Xtts.init_from_config(xtts_configuracion)
xtts_modelo.load_checkpoint(xtts_configuracion, checkpoint_dir="XTTS-v2", eval=True)
xtts_modelo.cuda().inference_stream()
hablante = "speakers/es_es/m4_script2_clean_segment_0.wav_es_2748.wav"

In [137]:
lista_audios = []
for segmento in lista_ciclica:
    texto = segmento['text']
    
    audio_data = xtts_modelo.synthesize(
                    texto,
                    xtts_configuracion,
                    speaker_wav=hablante,
                    gpt_cond_len=24,
                    temperature=0.4,
                    language='es',
                    speed=1.2
                )

    synthesized_audio = audio_data['wav']
    frecuencia_muestreo = xtts_configuracion.audio.sample_rate
    audio_array = np.array(synthesized_audio, dtype=np.float32)
    audio_array_int16 = np.int16(audio_array * 32767)
    audio = (audio_array_int16, len(audio_array) / frecuencia_muestreo)
    
    lista_audios.append(audio)

In [138]:
def encontrar_dispositivo_audio_salida(pyaudio_instancia):
        cantidad = pyaudio_instancia.get_device_count()
        for i in range(cantidad):
            info_dispositivo = pyaudio_instancia.get_device_info_by_index(i)
            if info_dispositivo['maxOutputChannels'] > 0:
                print(
                    "Usando el dispositivo de salida: {} - Índice: {}".format(info_dispositivo['name'], i))
                return i
        print("No se encontró ningún dispositivo de salida válido.")
        return None

In [139]:
def iniciar_stream(pyaudio_instancia, info_dispositivo=None):

        if info_dispositivo is None:
            info_dispositivo = encontrar_dispositivo_audio_salida(
                pyaudio_instancia)
        try:
            stream = pyaudio_instancia.open(
                format=pyaudio.paInt16, channels=1, rate=22050, output=True, output_device_index=info_dispositivo)
            return stream, pyaudio_instancia

        except Exception as e:
            print("error al iniciar stream")
            raise

In [140]:
pyaudio_instancia = pyaudio.PyAudio()


In [141]:

nuevo_stream, nuevo_pyaudio = iniciar_stream(pyaudio_instancia)

Usando el dispositivo de salida: Microsoft Sound Mapper - Output - Índice: 5


In [142]:
def escribir_audio(stream, data):
        bytes_per_chunk = 256
        for start in range(0, len(data), bytes_per_chunk):
            end = start + bytes_per_chunk
            chunk = data[start:end]
            stream.write(chunk.tobytes(), exception_on_underflow=False)

In [143]:
for audio in lista_audios:
    sound, _ = audio
    
    escribir_audio(nuevo_stream, sound)

## FORMA MODELO 2

In [100]:
texto_concatenado = ""

for segmento in lista_ciclica:
    texto1 = segmento['text']
    texto_concatenado += texto1

In [101]:
nombre_modelo = "tts_models/multilingual/multi-dataset/xtts_v2"
hablante = "speakers/es_es/m4_script2_clean_segment_0.wav_es_2748.wav"
lenguaje = "es"
dispositivo = 'cuda' if torch.cuda.is_available() else 'cpu'

In [102]:
tts = TTS(model_name=nombre_modelo)
tts.to(dispositivo)

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts


d:\Proyecto_holagrama_IA\LanGraph\v2email\lib\site-packages\torch\nn\modules\module.py:486: RuntimeWarning: coroutine 'play_audio' was never awaited
  super().__setattr__('_modules', OrderedDict())


TTS(
  (synthesizer): Synthesizer(
    (tts_model): Xtts(
      (gpt): GPT(
        (conditioning_encoder): ConditioningEncoder(
          (init): Conv1d(80, 1024, kernel_size=(1,), stride=(1,))
          (attn): Sequential(
            (0): AttentionBlock(
              (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
              (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
              (attention): QKVAttention()
              (x_proj): Identity()
              (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
            )
            (1): AttentionBlock(
              (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
              (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
              (attention): QKVAttention()
              (x_proj): Identity()
              (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
            )
            (2): AttentionBlock(
              (norm): GroupNorm32(32, 1024, eps=1e-05, affine=Tr

In [103]:
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16, channels=1,rate=22050, output=True)

In [104]:
audio_data = tts.tts(text=texto_concatenado, speaker_wav=hablante, language=lenguaje)

 > Text splitted to sentences.
['características de arrastrar y soltar, por ejemplo, que pueden enchufar y reproducir de inmediato y algunosla gente dice que es bastante complicado solo sé el 10% de React solo usouso estado y uso efectos en el 99% de los casos es más que suficiente para todas mis aplicacionesluego empaqueto todo eso en una aplicación Next.js es muy buena para enrutamiento de páginasy tener páginas agradables clasificadas en Google y también es super fácil deusar.', 'Para dar estilo a mis aplicaciones uso Tailwind CSS siento que es mucho más fácil que CSSporque el estilo se escribe directamente en las etiquetas HTML correctas, así que es fácil para míentender qué está sucediendo y, además, uso una biblioteca de interfaz de usuario llamada daisy UItiene un montón de componentes como botones, formularios, tarjetas que hacen que las cosas sean muchomás rápidas para enviar hay un montón de nuevas tendencias de diseño que surgen todos los días que']
[!] Warning: The text len

In [105]:
audio_array = np.array(audio_data, dtype=np.float32)
audio_array_int16 = np.int16(audio_array * 32767)

In [106]:
def _blocking_play_audio(stream, audio_data):
        audio_bytes = audio_data.tobytes()
        stream.write(audio_bytes)

In [107]:
async def play_audio(stream, audio_data):
        await asyncio.get_running_loop().run_in_executor(None, _blocking_play_audio, stream, audio_data)

In [108]:
async def play_audio2(stream, audio_data):
    await play_audio(stream, audio_data)